# Analyze Model Weights

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (20, 20), 'figure.dpi': 300}
)

## Load Weights

In [18]:
from common.data import get_model_weight

population = 'adhd'
measure = 'WISC_PSI'
age_group = 'all'

pls_weights = get_model_weight('pls', population, measure, age_group)
ridge_weights = get_model_weight('ridge', population, measure, age_group)

print(pls_weights.shape, ridge_weights.shape)

(34716,) (34716,)


## Intraclass Correlation (ICC)

In [19]:
%%time
import pingouin as pg

coefs = np.array([pls_weights, ridge_weights])
print(f'pls_{population}_{measure}_{age_group}', coefs.shape)

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

pls_adhd_WISC_PSI_all (2, 34716)
CPU times: user 29.9 s, sys: 160 ms, total: 30.1 s
Wall time: 30.1 s


,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.821,10.169,34715,34716,0.0,"[0.82, 0.82]"
ICC2,Single random raters,0.821,10.170,34715,34715,0.0,"[0.82, 0.82]"
ICC3,Single fixed raters,0.821,10.170,34715,34715,0.0,"[0.82, 0.82]"
ICC1k,Average raters absolute,0.902,10.169,34715,34716,0.0,"[0.9, 0.9]"
ICC2k,Average random raters,0.902,10.170,34715,34715,0.0,"[0.9, 0.9]"
ICC3k,Average fixed raters,0.902,10.170,34715,34715,0.0,"[0.9, 0.9]"
